In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import time
import datetime
import os

from sklearn.model_selection import train_test_split
from keras.models import load_model

import shap

c:\Users\Phine\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Reading de los modelos.

# Ruta donde están los modelos.

model_CDM = load_model(filepath = "../Versión 3 - actual/modelos/datasetCDM_2023_04_17_19_35")

model_GRD = load_model(filepath = "../Versión 3 - actual/modelos/datasetGRD_2023_04_17_20_53")

model_GRDT = load_model(filepath = "../Versión 3 - actual/modelos/datasetGRDT_2023_04_17_11_29")

model_SEV = load_model(filepath = "../Versión 3 - actual/modelos/datasetSEV_2023_04_17_22_09")

In [3]:
# start_code = time.time()

In [4]:
relative_1 = "F:/Universidad/Tesis/Codigo/Datos/Procesados"
x = pd.read_csv(f"{relative_1}/x_values.csv", dtype = int)
y_cdm = pd.read_csv(f"{relative_1}/y_dummies_cdm.csv", dtype = int)
y_grdt = pd.read_csv(f"{relative_1}/y_dummies_grdtipo.csv", dtype = int)
y_grd = pd.read_csv(f"{relative_1}/y_dummies_grd.csv", dtype = int)
y_sev = pd.read_csv(f"{relative_1}/y_dummies_sev.csv", dtype = int)
df = pd.read_csv(f'{relative_1}/dataset_procesado_pbi.csv', dtype = str)

In [5]:
lst_y = [y_cdm, y_grdt, y_grd, y_sev]
lst_y_str = ['CDM', 'GRDT', 'GRD', 'SEV']
lst_model = [model_CDM, model_GRDT, model_GRD, model_SEV]

In [6]:
x_train, x_test, y_train, _ = train_test_split(x, lst_y[0], train_size = 0.0004, test_size = 0.0004, random_state = 0, stratify = lst_y[0])

lista_indices_train = x_train.index.to_list()
lista_indices_test = x_test.index.to_list()

df_recortado = df.iloc[lista_indices_train]

In [229]:
y_test_cdm = y_cdm.iloc[lista_indices_test]
y_test_grdt = y_grdt.iloc[lista_indices_test]
y_test_grd = y_grd.iloc[lista_indices_test]
y_test_sev = y_sev.iloc[lista_indices_test]

y_test_models = [y_test_cdm, y_test_grdt, y_test_grd, y_test_sev]

In [33]:
i = 3

In [34]:
model = lst_model[i]

In [35]:
def predict_proba(X):
    return model.predict(X, verbose = False)

In [ ]:
# Crear un objeto KernelExplainer de SHAP
background_data = shap.sample(x_train, 100)  # Usar una muestra del conjunto de entrenamiento como datos de fondo
explainer = shap.KernelExplainer(predict_proba, background_data)
lst_explainers.append(explainer)
# Calcular los valores SHAP para el conjunto de datos de prueba
shap_values = explainer.shap_values(x_test, nsamples = 100)
lst_shap_values.append(shap_values)

In [41]:
type_plot = "BAR_PLOT"
y_str = ['CDM', 'GRDT', 'GRD', 'SEV']

In [48]:
i = 0

In [181]:
# Establece la resolución de la figura
plt.rcParams["figure.dpi"] = 300
for n in range(len(lst_shap_values[i])):
    # Crea una nueva figura con el tamaño deseado
    plt.figure(figsize=(12, 5))
    plt.title("Importancia de datos de entrada modelo CDM")
    # Genera el gráfico de resumen
    shap.summary_plot(lst_shap_values[i][n], x_test, plot_type='bar', show = False)
    output_folder = f"F:/Universidad/Tesis/Codigo/Visualización de datos/SHAP/Summary Plot/{y_str[i]}"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    plt.savefig(f"{output_folder}/class_{y_train_models[i].columns[n]}.png")
    plt.close()  # Cierra la figura para liberar memoria

In [234]:
i = 3

In [239]:
lst_predictions = []

In [240]:
for asd in range(len(lst_model)):
    prediccion = lst_model[asd].predict(x_test)
    lst_predictions.append(prediccion)

24/24 [==============================] - 0s 2ms/step


In [256]:
lst_predictions[i][0]

0.744962

In [270]:
row_index = 0

In [271]:
plt.figure(figsize = [18, 15], dpi = 300)
def class_labels(row_index):
    return [f'Clase {y_test_models[i].columns[f]} ({lst_predictions[i][row_index, f].round(2):.2f})' for f in range(len(y_test_models[i].columns))]

shap.multioutput_decision_plot(list(lst_explainers[i].expected_value), list(lst_shap_values[i]),
                                row_index = row_index,
                                feature_names=list(x_test.columns),
                                highlight=[np.argmax(lst_predictions[i][row_index])],
                                legend_labels=class_labels(row_index),
                                legend_location='lower right',
                                auto_size_plot = False,
                                show = False)
# Guardar cada gráfico en un archivo separado
output_folder = f"F:/Universidad/Tesis/Codigo/Visualización de datos/SHAP/prueba1"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
plt.savefig(f"{output_folder}/{y_str[i]}_multioutput_decision_plot_patient_{x_test['DIAGNOSTICO1'].index[row_index]}.png")
# Cerrar la figura para liberar memoria
plt.close()

In [155]:
shap.force_plot(lst_explainers[0].expected_value[0], lst_shap_values[0][0][1], x_test.iloc[1, :], show = True)

In [68]:
class_shap_values = lst_shap_values[i]
random_shap_values = class_shap_values[0]
# Crear una figura para cada clase
plt.figure(figsize = [15, 15], dpi = 150)
# Generar el bar_plot para la clase actual y el valor SHAP seleccionado
shap.plots.scatter(
    lst_explainers[0],
    show=True  # Establecer en False para evitar que se muestre la trama en la ventana de salida
)

# Guardar cada gráfico en un archivo separado
# output_folder = f"F:/Universidad/Tesis/Codigo/Visualización de datos/SHAP_segundo_intento"
# if not os.path.exists(output_folder):
#     os.makedirs(output_folder)
# plt.savefig(f"{output_folder}/{type_plot}_{y_str[i]}_class_{y_train_models[i].columns[n]}_patient_{x_train['DIAGNOSTICO1'].index[j]}.png")

# Cerrar la figura para liberar memoria
plt.close()

AssertionError: The shap_values paramemter must be a shap.Explanation object!

<Figure size 2250x2250 with 0 Axes>

In [49]:
for n, class_shap_values in enumerate(lst_shap_values[i]):
    start_time = time.time()
    print(f"Obteniendo gráficas de la clase: {y_train_models[i].columns[n]}")
    for j in range(100):
        # Seleccionar una fila de los valores SHAP
        random_shap_values = class_shap_values[j, :]

        # Crear una figura para cada clase
        plt.figure(figsize = [15, 15], dpi = 150)

        # Generar el bar_plot para la clase actual y el valor SHAP seleccionado
        shap.bar_plot(
            random_shap_values,
            feature_names=x_test.columns,
            max_display=25,
            show=False  # Establecer en False para evitar que se muestre la trama en la ventana de salida
        )

        # Guardar cada gráfico en un archivo separado
        output_folder = f"F:/Universidad/Tesis/Codigo/Visualización de datos/SHAP_segundo_intento"
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
        plt.savefig(f"{output_folder}/{type_plot}_{y_str[i]}_class_{y_train_models[i].columns[n]}_patient_{x_test['DIAGNOSTICO1'].index[j]}.png")

        # Cerrar la figura para liberar memoria
        plt.close()

    end_time = time.time()
    celda_time = end_time - start_time
    minutes, seconds = divmod(celda_time, 60)
    print(f"Graficas obtenidas, Tiempo total de ejecución: {int(minutes)}:{int(seconds)} minutos.")
    print("--------------------------\n")

Obteniendo gráficas de la clase: 1
Graficas obtenidas, Tiempo total de ejecución: 0:30 minutos.
--------------------------

Obteniendo gráficas de la clase: 2
Graficas obtenidas, Tiempo total de ejecución: 0:29 minutos.
--------------------------

Obteniendo gráficas de la clase: 3
Graficas obtenidas, Tiempo total de ejecución: 0:30 minutos.
--------------------------

Obteniendo gráficas de la clase: 4
Graficas obtenidas, Tiempo total de ejecución: 0:30 minutos.
--------------------------

Obteniendo gráficas de la clase: 5
Graficas obtenidas, Tiempo total de ejecución: 0:29 minutos.
--------------------------

Obteniendo gráficas de la clase: 6
Graficas obtenidas, Tiempo total de ejecución: 0:29 minutos.
--------------------------

Obteniendo gráficas de la clase: 7
Graficas obtenidas, Tiempo total de ejecución: 0:29 minutos.
--------------------------

Obteniendo gráficas de la clase: 8
Graficas obtenidas, Tiempo total de ejecución: 0:29 minutos.
--------------------------

Obtenien

In [8]:
# for i in range(len(lst_model)):
#     shap_function(i)

# end_code = time.time()
# f_time = end_code - start_code
# td = datetime.timedelta(seconds = f_time)
# dt = datetime.datetime(2023, 5, 1, 0, 0, 0)
# dt_final = dt + td
# hora_final = dt_final.strftime("%H:%M:%S")

# print(f"Tiempo total de ejecución: {hora_final} Hrs.")